In [40]:
import numpy as np
import pandas as pd
import random
from collections import OrderedDict, defaultdict
import multiprocessing as mp
import glob
import sys,os
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm
tqdm.pandas()
from gensim.models import Word2Vec

In [2]:
with gezi.Timer('read train user_actions'):
  d = pd.read_csv('../input/user_action.csv')
  try:
    d1 = pd.read_csv('../input/v1/user_action.csv')
    d = pd.concat([d, d1])
  except Exception:
    pass

read train user_actions start
read train user_actions duration: 3.6904406547546387


In [3]:
with gezi.Timer('read test and merge'):
  dt = pd.read_csv('../input/test_a.csv')
  try:
    dt1a = pd.read_csv('../input/v1/test_a.csv')
    dt1b = pd.read_csv('../input/v1/test_b.csv')
    dt = pd.concat([dt, dt1a, dt1b])
  except Exception:
    dtb = pd.read_csv('../input/test_b.csv')
    dt = pd.concat([dt, dtb])

read test and merge start
read test and merge duration: 0.12495541572570801


In [4]:
dt['date_'] = 15

In [5]:
set(d.date_)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

In [6]:
dshow = pd.concat([d[['userid', 'feedid', 'date_']],
                  dt[['userid', 'feedid', 'date_']]])
dshow = dshow.sort_values(['date_'], ascending=True)

In [7]:
doc_vocab = gezi.Vocab('../input/doc_vocab.txt')

In [8]:
user_vocab = gezi.Vocab('../input/user_vocab.txt')

In [9]:
du = dshow.groupby(['userid'])['feedid'].progress_apply(list).reset_index(name='docs')

  0%|          | 0/20000 [00:00<?, ?it/s]

In [10]:
du.head()

,userid,docs
0,8,"[71474, 56747, 105148, 25619, 48027, 83826, 40..."
1,12,"[69176, 102465, 85004, 36851, 106104, 80434, 3..."
2,13,"[15729, 60767, 23302, 31961, 52458, 30296, 796..."
3,25,"[49745, 79014, 2820, 49065, 69321, 104284, 846..."
4,37,"[10705, 32160, 60130, 2505, 18148, 49725, 4113..."


In [11]:
dd = dshow.groupby(['feedid'])['userid'].progress_apply(list).reset_index(name='users')

  0%|          | 0/99420 [00:00<?, ?it/s]

In [12]:
dd.head()

,feedid,users
0,0,"[147722, 73624]"
1,1,"[27024, 33584, 194804, 42126, 182589, 65828, 1..."
2,3,"[157285, 152350, 235809, 9979]"
3,4,"[187003, 5534, 104470, 154220, 33844, 219000, ..."
4,6,"[15685, 187003]"


In [27]:
dic = {}

In [28]:
for row in tqdm(du.itertuples()):
  dic[f'u{row.userid}'] = [f'd{x}' for x in row.docs]

0it [00:00, ?it/s]

In [29]:
for row in tqdm(dd.itertuples()):
  dic[f'd{row.feedid}'] = [f'u{x}' for x in row.users]

0it [00:00, ?it/s]

In [31]:
# dic

In [52]:
window = 10
emb_dim = 128
path_len = 20
rounds = 2

In [56]:
from gensim.models.callbacks import CallbackAny2Vec

class MonitorCallback(CallbackAny2Vec):
  def __init__(self, name):
    self.name = name
    self.epoch = 1
    self.timer = gezi.Timer()
    
  def on_epoch_end(self, model):
    # TODO 为什么打印train loss一直是0
    print('name:', self.name, 'epoch:', self.epoch, "model loss:", model.get_latest_training_loss(), f'elapsed minutes: {self.timer.elapsed_minutes():.2f}')  # print loss
    self.epoch += 1

In [55]:
sentences=[]
length=[]
for round in tqdm(range(rounds)):
  for key in tqdm(dic, total=len(dic)):
    sentence=[key]
    while len(sentence) != path_len:
      key=dic[sentence[-1]][random.randint(0, len(dic[sentence[-1]])-1)]
      if len(sentence)>=2 and key == sentence[-2]:
        break
      else:
        sentence.append(key)
    sentences.append(sentence)
  length.append(len(sentence))
print(np.mean(length))
print(len(sentences))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/119420 [00:00<?, ?it/s]

  0%|          | 0/119420 [00:00<?, ?it/s]

Exception ignored in: <bound method tqdm.__del__ of <tqdm.auto.tqdm object at 0x7f9714078400>>
Traceback (most recent call last):
  File "/home/huigecheng/.local/lib/python3.6/site-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/home/huigecheng/.local/lib/python3.6/site-packages/tqdm/notebook.py", line 278, in close
    self.disp(bar_style='success')
AttributeError: 'tqdm' object has no attribute 'disp'


20.0
238840


In [57]:
random.shuffle(sentences)
monitor = MonitorCallback('deepwalk')
w2v = Word2Vec(sentences, vector_size=emb_dim, sg=1, window=window, workers=mp.cpu_count(), min_count=1, epochs=10, callbacks=[monitor])

name: deepwalk epoch: 1 model loss: 0.0 elapsed minutes: 0.23
name: deepwalk epoch: 2 model loss: 0.0 elapsed minutes: 0.14
name: deepwalk epoch: 3 model loss: 0.0 elapsed minutes: 0.14
name: deepwalk epoch: 4 model loss: 0.0 elapsed minutes: 0.14
name: deepwalk epoch: 5 model loss: 0.0 elapsed minutes: 0.14
name: deepwalk epoch: 6 model loss: 0.0 elapsed minutes: 0.12
name: deepwalk epoch: 7 model loss: 0.0 elapsed minutes: 0.13
name: deepwalk epoch: 8 model loss: 0.0 elapsed minutes: 0.14
name: deepwalk epoch: 9 model loss: 0.0 elapsed minutes: 0.13
name: deepwalk epoch: 10 model loss: 0.0 elapsed minutes: 0.17


In [58]:
uvocab = gezi.Vocab('../input/user_vocab.txt')
dvocab = gezi.Vocab('../input/doc_vocab.txt')
uemb = np.zeros([uvocab.size(), emb_dim])
for i in range(uvocab.size()):
  word = f'u{uvocab.key(i)}'
  if word in w2v.wv:
    uemb[i] = w2v.wv[word]
demb = np.zeros([dvocab.size(), emb_dim])
for i in range(dvocab.size()):
  word = f'd{dvocab.key(i)}'
  if word in w2v.wv:
    demb[i] = w2v.wv[word]

In [59]:
np.save('../input/15/user_deepwalk_emb.npy', uemb)
np.save('../input/15/doc_deepwalk_emb.npy', demb)